In [0]:
# обновить файлы .py и скопировать остальные файлы в outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

Обратите внимание на одно замечание по поводу функции `dataset.make_one_shot_iterator`.

Сейчас скрипт работает с  TensorFlow 2.0:

Обратите внимание, что из-за модуля `tf.compat.v1`, сконвертированный скрипт также будет запускаться в TensorFlow 1.14. 

In [0]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

## Детальный отчет

Скрипт также публикует подробный список изменений. В этом примере он нашел одну возможно небезопасную трансформацию и добавил предупреждение в начало файла: 

In [0]:
!head -n 20 tree_report.txt

Обратите внимание вновь на одно замечание о `Dataset.make_one_shot_iterator function`.

В остальных случаях результат объяснит причину для нетривиальных изменений:

In [0]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

In [0]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

In [0]:
!cat dropout_report.txt

Вот измененное содержимое файла, обратите внимание, как скрипт добавляет имена аргументов для работы с перемещенными и переименованными аргументами:

In [0]:
!cat dropout_v2.py

Больший проект может содержать мало ошибок. Например, конвертируеме модель deeplab:

In [0]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

Это сгенерировало выходные файлы:

In [0]:
!ls deeplab_v2

Но там были ошибки. Отчет поможет вам точно определить, что нужно исправить, прежде чем запускать скрипт. Вот первые три ошибки:

In [0]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

## "Безопасный" режим

У скрипт конвертации есть также менее инвазивный `БЕЗОПАСНЫЙ` режим который просто меняет импорты для использования модуля `tensorflow.compat.v1`:

In [0]:
!cat dropout.py

In [0]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

In [0]:
!cat dropout_v2_safe.py

Как вы можете видеть это не обновляет ваш код, но позволяет TensorFlow 1 коду запускаться в TensorFlow 2

## Предостережения

- Не обновляйте части вашего кода вручную перед запуском скрипта. В частности, функции с переупорядоченными аргументами, такие как `tf.argmax` или` tf.batch_to_space`, вынудят скрипт неправильно добавить аргументы ключевых слов, что запутает ваш существующий код.

- Скрипт предполагает, что `tensorflow` импортирован с использованием `import tensorflow as tf`.

- Скрипт не переупорядочивает аргументы. Вместо этого скрипт добавляет ключевые слова аргументов к функциям у которых аргументы поменяли порядок.

- Проверьте [tf2up.ml](http://tf2up.ml) для подходящего инструмента обновления ноутбуков Jupyter
  и файлов Python в репозитории GitHub.

Чтобы сообщить об ошибках в скрипте обновления или отправить запрос на добавление функции, отправьте сообщение об ошибке на [GitHub](https://github.com/tensorflow/tensorflow/issues). И если вы тестируете TensorFlow 2.0, мы хотим знать об этом! Вступайте в [сообщество тестирования TF 2.0](https://groups.google.com/a/tensorflow.org/forum/#!forum/testing) и отправляйте вопросы и обсуждения на [testing@tensorflow.org](mailto:testing@tensorflow.org).